# Paper Trading

Deploy strategy to paper trading with LiveEngine, LiveDataStore, and simulated order execution.

In [18]:
from datetime import datetime, timezone, timedelta
from clyptq import CostModel, Constraints, EngineMode
from clyptq.data.stores.live_store import LiveDataStore
from clyptq.data.loaders.ccxt import CCXTLoader
from clyptq.trading.engine import LiveEngine
from clyptq.trading.execution import LiveExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.portfolio.constructors import TopNConstructor
from clyptq.trading.strategy.base import SimpleStrategy

## 1. Define Strategy

Same strategy from backtesting

In [19]:
class MomentumStrategy(SimpleStrategy):
    def __init__(self):
        super().__init__(
            factors_list=[MomentumFactor(lookback=30), VolatilityFactor(lookback=30)],
            constructor=TopNConstructor(top_n=3),
            constraints_obj=Constraints(
                max_position_size=0.40,
                max_gross_exposure=1.0,
                min_position_size=0.10,
                max_num_positions=5,
            ),
            schedule_str="weekly",
            warmup=35,
            name="Momentum",
        )

strategy = MomentumStrategy()
print(f"Strategy: {strategy.name}")
print(f"Rebalance: {strategy.schedule()}")
print(f"Warmup: {strategy.warmup_periods()} days")

Strategy: Momentum
Rebalance: weekly
Warmup: 35 days


## 2. Set up LiveDataStore

Rolling window data management for real-time trading.

In [ ]:
# Universe
symbols = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "ADA/USDT"]

# LiveDataStore with 60-day rolling window
live_store = LiveDataStore(lookback_days=60)

print(f"Universe: {len(symbols)} symbols")
print(f"Rolling window: {live_store.lookback_days} days")
print("\nLoading historical data for warmup...")

# Load warmup data using CCXTLoader
loader = CCXTLoader(exchange_id="binance")

for symbol in symbols:
    print(f"  Loading {symbol}...")
    df = loader.load_ohlcv(
        symbol=symbol,
        timeframe="1d",
        limit=60,  # 60 days warmup
    )
    live_store.add_historical(symbol, df)

print("\nWarmup data loaded")
print(f"Symbols loaded: {live_store.num_symbols()}")

## 3. Set up LiveExecutor (Paper Mode)

Simulates order execution without real trades.

In [ ]:
# Paper mode executor (no real orders)
executor = LiveExecutor(
    exchange_id="binance",
    api_key="",  # Not needed in paper mode
    api_secret="",
    paper_mode=True,  # Critical: paper trading
    cost_model=CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0),
)

print("Executor: Paper mode")
print("No real orders will be sent")
print("Simulated fills at market prices")

## 4. Create LiveEngine

Initialize LiveEngine in paper mode.

In [22]:
engine = LiveEngine(
    strategy=strategy,
    data_store=live_store,
    executor=executor,
    initial_capital=10000.0,
    mode=EngineMode.PAPER,
)

print("LiveEngine created")
print(f"Mode: {engine.mode}")
print(f"Initial capital: ${engine.portfolio.cash:,.2f}")
print(f"Universe: {len(symbols)} symbols")

LiveEngine created
Mode: EngineMode.PAPER
Initial capital: $10,000.00
Universe: 5 symbols


## 5. Simulate One Trading Step

Synchronous execution at specific timestamp using step().

In [23]:
# Get latest prices from exchange
print("Fetching current prices...")
current_prices = {}
for symbol in symbols:
    ticker = loader.exchange.fetch_ticker(symbol)
    current_prices[symbol] = ticker['last']
    print(f"  {symbol}: ${ticker['last']:,.2f}")

# Current timestamp
now = datetime.now(timezone.utc)
print(f"\nTimestamp: {now}")

# Execute one step
print("\nExecuting trading step...")
result = engine.step(timestamp=now, prices=current_prices)

print("\nEXECUTION RESULT")
print("=" * 80)
print(f"Action: {result.action}")
print(f"Rebalance reason: {result.rebalance_reason or 'N/A'}")
print(f"Orders generated: {len(result.orders)}")
print(f"Fills executed: {len(result.fills)}")

if result.fills:
    print("\nFills:")
    for fill in result.fills:
        print(f"  {fill.side.value:<4} {fill.symbol:<12} {fill.amount:>10.6f} @ ${fill.price:>10.2f}")

# Portfolio snapshot
snapshot = result.snapshot
print(f"\nPortfolio Snapshot:")
print(f"  Equity: ${snapshot.equity:,.2f}")
print(f"  Cash: ${snapshot.cash:,.2f}")
print(f"  Positions: {len(snapshot.positions)}")

if snapshot.positions:
    print("\nPositions:")
    for symbol, position in snapshot.positions.items():
        market_value = position.amount * current_prices[symbol]
        weight = market_value / snapshot.equity if snapshot.equity > 0 else 0.0
        print(
            f"  {symbol:<12} "
            f"qty={position.amount:>10.6f} "
            f"value=${market_value:>10.2f} "
            f"weight={weight:>6.2%}"
        )

Fetching current prices...
  BTC/USDT: $91,152.07
  ETH/USDT: $3,136.84
  BNB/USDT: $886.83
  SOL/USDT: $134.31
  ADA/USDT: $0.40

Timestamp: 2026-01-04 12:56:07.786676+00:00

Executing trading step...

EXECUTION RESULT
Action: skip
Rebalance reason: no_scores
Orders generated: 0
Fills executed: 0

Portfolio Snapshot:
  Equity: $10,000.00
  Cash: $10,000.00
  Positions: 0


## 6. Continuous Trading Loop

Production pattern runs in infinite loop with schedule checking and risk monitoring.

## 7. Transition to Live Trading

Change executor to live mode and switch engine to EngineMode.LIVE when ready for real money.

## 8. Production Checklist

Risk Management:
- Define max drawdown limit
- Set position size limits
- Implement stop-loss
- Set up kill switch

Monitoring:
- Log all trades
- Track P&L in real-time
- Set up alerts (email/SMS)
- Monitor position drift

Operational:
- Secure API keys (environment variables)
- Test on sandbox/testnet first
- Set up backup server
- Document recovery procedures

Testing:
- Paper trade for 1 month
- Verify position reconciliation
- Test order execution quality
- Validate factor computation

## 9. Key Differences: Backtest vs Paper vs Live

| Aspect | Backtest | Paper | Live |
|--------|----------|-------|------|
| Data | Historical | Real-time | Real-time |
| Orders | Simulated | Simulated | Real |
| Speed | Fast | Real-time | Real-time |
| Risk | None | None | Real money |
| Slippage | Modeled | Modeled | Actual |
| Engine | BacktestEngine | LiveEngine | LiveEngine |
| Store | DataStore | LiveDataStore | LiveDataStore |
| Executor | BacktestExecutor | LiveExecutor(paper=True) | LiveExecutor(paper=False) |

## Summary

Load historical data for warmup, create LiveDataStore with rolling window, set up LiveExecutor in paper mode, create LiveEngine with PAPER mode, run step() at scheduled intervals, monitor positions and P&L, then switch to LIVE mode after validation.